In [2]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import pygame
import numpy as np
from pygame import mixer

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalFace_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [4]:
model = load_model('Models//model.h5')

In [5]:
mixer.init()
sound= mixer.Sound(r'C:\Users\parth\Downloads\Fall 2022\CSIS 4260 - Special Topics in Data Analytics\CSIS 4260 - Project\alarm.wav')
cap = cv2.VideoCapture(0)
score = 0
while True:
    ret, frame = cap.read()
    height,width = frame.shape[0:2]
#     converting the color to grayscale frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.2, minNeighbors = 3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 1)
    
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color=(255,0,0), thickness = 3)
    
    for (ex,ey,ew,eh) in eyes:
#         cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color=(0,255,0), thickness = 3)
        # preprocessing steps
        eye = frame[ey:ey+eh,ex:ex+w]
        eye = cv2.resize(eye,(80,80))
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye,axis=0)
        
        # Model Prediction
        prediction = model.predict(eye)
        
        # Closed Eyes
        if prediction[0][0] > 0.50:
            cv2.putText(frame,'Closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=0.75,
                        color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score ' + str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=0.5,
                        color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            score = score + 1
            if(score>8):
                try:
                    sound.play()
                except:
                    pass
        # Open Eyes
        elif prediction[0][1] > 0.50:
            cv2.putText(frame,'Open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=0.75,
                        color=(255,255,255),thickness=1)
            cv2.putText(frame,'Score ' + str(score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX,fontScale=0.5,
                        color=(255,255,255),thickness=1)
            score=score-1
            if(score < 0):
                score = 0
    cv2.imshow('frame',frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()        
#         BGR format for the color of the traingle
        

1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 27ms/step


KeyboardInterrupt: 